In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle
import os

In [2]:
words = None
if False:
    from nltk.corpus import gutenberg as corpus
    words = corpus.words()
    vocabulary = vocabulary[803:-2]
    # first 803 and last 2 words are punctuation signs, numbers and underscored
    # words like _home_
else:
    # Using the whole corpus means too much work to later compute the words at a
    # distance for each word in the vocabulary, so we load just one book
    import nltk
    nltk.download('gutenberg')
    words = nltk.corpus.gutenberg.words('austen-emma.txt')
    # using all the words in the book will take 25-30 min. to process later
    # so we limit its number for the moment.
    words = words[:10000] # cell [15] will take 1.5 min.
    #words = words[:100000] # cell [15] will take 15 min.


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [3]:
vocabulary = list(set(words))
vocabulary.sort()
# get rid of some non-words like ',' '--' '['
idx_first_word = vocabulary.index('A')
vocabulary = vocabulary[idx_first_word:]
# plus some more annoying non-words
vocabulary.remove('[')
vocabulary.remove(']')
vocabulary.remove('`')
vocabulary.remove('II')
vocabulary = np.array(vocabulary)
print(vocabulary)
print('Vocabulary size {} words'.format(len(vocabulary)))

['A' 'Abbey' 'After' ... 'yourself' 'youth' 'youthful']
Vocabulary size 1711 words


For each word in the vocabulary find the nearest words = at Levenshtein distance up to ``MAX_DIST``. This is a long computation, $O(n^2)$ for $n$ size of the vocabulary. We try to speed up it a little : if $\text{dist}(w_1, w_2) \leq d$ then $|\text{len}(w_1) - \text{len}(w_2)| \leq d$. This reduces the candidate words in the vocabulary for which to compute the distance to each word.
We save the resulting dictionary to avoid recomputing it each time.

In [4]:
from nltk.metrics.distance import edit_distance

def levenshtein(s1, s2):
    return edit_distance(s1, s2, substitution_cost=1, transpositions=True)

word_lengths = np.array([len(w) for w in vocabulary])
dict_lengths = {}
for l in range(min(word_lengths), max(word_lengths)+1):
    dict_lengths[l] = vocabulary[word_lengths==l] # needs vocabulary to be a numpy array

min_length = min(dict_lengths.keys())
max_length = max(dict_lengths.keys())

MAX_DIST = 1
fname = 'close_words_{}.pkl'.format(MAX_DIST)
if not os.path.exists(fname):
    close_words = {}
    for word in tqdm(vocabulary):
        length = len(word)
        candidate_words = []
        d1 = max(min_length, length - MAX_DIST)
        d2 = min(max_length, length + MAX_DIST)
        for d in range(d1, d2+1):
            candidate_words.extend(dict_lengths[d])
        close_words[word] = [w for w in candidate_words if levenshtein(word,w) <= MAX_DIST]

    with open(fname,'wb') as f:
        pickle.dump(close_words, f)
else:
    close_words = pickle.load(open(fname,'rb'))

100%|██████████| 1711/1711 [00:59<00:00, 28.72it/s]


Given one sentence $X$, which is a list of words, build the candidates to correct sentence $C(X)$ assuming at most one word is mispelled.

In [60]:
#sentence = 'Only two of the apples'
sentence = 'I wish you where here'
X = sentence.split(' ')
for x in X:
  assert x in vocabulary, 'All the words in the sentence must belong to the '\
      + 'vocabulary, {} doesn\'t'.format(x)
CX = [X] # no errors
for i in range(len(X)): # one mispelled word at a time
    if X[i] in vocabulary:
        for cw in close_words[X[i]]:
            if cw != X[i]:
                C = X.copy()
                C[i] = cw
                CX.append(C)
    else:
        pass # let it be as is
for W in CX:
    print('\t'.join(W))

I	wish	you	where	here
A	wish	you	where	here
a	wish	you	where	here
s	wish	you	where	here
IV	wish	you	where	here
If	wish	you	where	here
In	wish	you	where	here
It	wish	you	where	here
I	fish	you	where	here
I	with	you	where	here
I	wish	You	where	here
I	wish	your	where	here
I	wish	you	here	here
I	wish	you	were	here
I	wish	you	There	here
I	wish	you	Where	here
I	wish	you	there	here
I	wish	you	where	her
I	wish	you	where	hers
I	wish	you	where	were
I	wish	you	where	There
I	wish	you	where	Where
I	wish	you	where	there
I	wish	you	where	where


In [95]:
#ex2
sentence = 'I wish you where here'
X = sentence.split(' ')

for x in X:
    assert x in vocabulary, 'All the words in the sentence must belong to the vocabulary, {} doesn\'t'.format(x)

CX = [X]  # Original sentence

for i in range(len(X)):
    if X[i] in vocabulary:
        for cw1 in close_words[X[i]]:
            if cw1 != X[i]:
                C1 = X.copy()
                C1[i] = cw1
                CX.append(C1)

                for j in range(i+1, len(X)):
                    if X[j] in vocabulary and X[j] != cw1:
                        for cw2 in close_words[X[j]]:
                            if cw2 != X[j]:
                                C2 = C1.copy()
                                C2[j] = cw2
                                CX.append(C2)
    else:
        pass

for W in CX:
    print('\t'.join(W))


I	wish	you	where	here
A	wish	you	where	here
A	fish	you	where	here
A	with	you	where	here
A	wish	You	where	here
A	wish	your	where	here
A	wish	you	here	here
A	wish	you	were	here
A	wish	you	There	here
A	wish	you	Where	here
A	wish	you	there	here
A	wish	you	where	her
A	wish	you	where	hers
A	wish	you	where	were
A	wish	you	where	There
A	wish	you	where	Where
A	wish	you	where	there
A	wish	you	where	where
a	wish	you	where	here
a	fish	you	where	here
a	with	you	where	here
a	wish	You	where	here
a	wish	your	where	here
a	wish	you	here	here
a	wish	you	were	here
a	wish	you	There	here
a	wish	you	Where	here
a	wish	you	there	here
a	wish	you	where	her
a	wish	you	where	hers
a	wish	you	where	were
a	wish	you	where	There
a	wish	you	where	Where
a	wish	you	where	there
a	wish	you	where	where
s	wish	you	where	here
s	fish	you	where	here
s	with	you	where	here
s	wish	You	where	here
s	wish	your	where	here
s	wish	you	here	here
s	wish	you	were	here
s	wish	you	There	here
s	wish	you	Where	here
s	wish	you	there	here
s	wish	

Likelihood $P(X | W) = \prod_{i=1}^n p(x_i | w_i)$ where $n$ is number of words in $X$ (same as in $W$), and $p(x | w)$  is Eq. B.8. $X$ is the written sentence, $W$ are the candidate sentences in $C(X)$. Each $W$ contains zero (ie, $W=X$) or at most one mispelled word, and in this case the mispelled word $w_i$ is at a Levenshtein distance $1...$ ``MAX_DIST`` of the written word $x_i$

In [61]:
alpha = 0.95
likelihoods = []
for W in CX:
    PXW = 1.0
    #print(X,W)
    for x,w in zip(X,W):
        if w==x:
            pxw = alpha
        else:
            close_to_x = close_words[x] # includes x itself
            pxw = (1-alpha) / (len(close_to_x) - 1) # so we substract 1
        PXW *= pxw
    likelihoods.append(PXW)

likelihoods = np.array(likelihoods)
idx_most_likely = likelihoods.argmax()
print('Sentence with highest likelihood is the written one, X')
print('\t'.join(CX[idx_most_likely]), '\t', likelihoods[idx_most_likely])
print('Likelihoods')
num_candidates = len(CX)
for i in range(num_candidates):
    print('\t'.join(CX[i]), '\t', likelihoods[i])


Sentence with highest likelihood is the written one, X
I	wish	you	where	here 	 0.7737809374999999
Likelihoods
I	wish	you	where	here 	 0.7737809374999999
A	wish	you	where	here 	 0.005817901785714291
a	wish	you	where	here 	 0.005817901785714291
s	wish	you	where	here 	 0.005817901785714291
IV	wish	you	where	here 	 0.005817901785714291
If	wish	you	where	here 	 0.005817901785714291
In	wish	you	where	here 	 0.005817901785714291
It	wish	you	where	here 	 0.005817901785714291
I	fish	you	where	here 	 0.020362656250000017
I	with	you	where	here 	 0.020362656250000017
I	wish	You	where	here 	 0.020362656250000017
I	wish	your	where	here 	 0.020362656250000017
I	wish	you	here	here 	 0.008145062500000006
I	wish	you	were	here 	 0.008145062500000006
I	wish	you	There	here 	 0.008145062500000006
I	wish	you	Where	here 	 0.008145062500000006
I	wish	you	there	here 	 0.008145062500000006
I	wish	you	where	her 	 0.00581790178571429
I	wish	you	where	hers 	 0.00581790178571429
I	wish	you	where	were 	 0.00581790178

Priors $P(W)$ for all $W \in C(X)$ computed by a LM model, for instance tri-grams (on this same corpus). We've chosen the stupid backoff version.

Higher values of α assign more weight to the likelihood of correct words, while lower values of α assign more weight to the likelihood of misspelled words

In [25]:
#ex3
alpha = 0.8
likelihoods = []
for W in CX:
    PXW = 1.0
    #print(X,W)
    for x,w in zip(X,W):
        if w==x:
            pxw = alpha
        else:
            close_to_x = close_words[x] # includes x itself
            pxw = (1-alpha) / (len(close_to_x) - 1) # so we substract 1
        PXW *= pxw
    likelihoods.append(PXW)

likelihoods = np.array(likelihoods)
idx_most_likely = likelihoods.argmax()
print('Sentence with highest likelihood is the written one, X')
print('\t'.join(CX[idx_most_likely]), '\t', likelihoods[idx_most_likely])
print('Likelihoods')
num_candidates = len(CX)
for i in range(num_candidates):
    print('\t'.join(CX[i]), '\t', likelihoods[i])

Sentence with highest likelihood is the written one, X
I	wish	you	where	here 	 0.32768000000000014
Likelihoods
I	wish	you	where	here 	 0.32768000000000014
A	wish	you	where	here 	 0.011702857142857143
a	wish	you	where	here 	 0.011702857142857143
s	wish	you	where	here 	 0.011702857142857143
IV	wish	you	where	here 	 0.011702857142857143
If	wish	you	where	here 	 0.011702857142857143
In	wish	you	where	here 	 0.011702857142857143
It	wish	you	where	here 	 0.011702857142857143
I	fish	you	where	here 	 0.040959999999999996
I	with	you	where	here 	 0.040959999999999996
I	wish	You	where	here 	 0.04096
I	wish	your	where	here 	 0.04096
I	wish	you	here	here 	 0.016384000000000003
I	wish	you	were	here 	 0.016384000000000003
I	wish	you	There	here 	 0.016384000000000003
I	wish	you	Where	here 	 0.016384000000000003
I	wish	you	there	here 	 0.016384000000000003
I	wish	you	where	her 	 0.011702857142857143
I	wish	you	where	hers 	 0.011702857142857143
I	wish	you	where	were 	 0.011702857142857143
I	wish	you	whe

In [7]:
nltk.download('punkt')

sents = nltk.corpus.gutenberg.sents('austen-emma.txt')
text = []
for s in tqdm(sents):
    text.append(s[:-1]) # except ending point

from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm.models import StupidBackoff

n=3
lm = StupidBackoff(alpha=0.4, order=n)
train, vocab = padded_everygram_pipeline(n, text)
lm.fit(train, vocab)
print(lm.counts)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
100%|██████████| 7752/7752 [00:00<00:00, 14076.98it/s]


<NgramCounter with 3 ngram orders and 623964 ngrams>


In [8]:
def P(W, verbose=False):
    S = ['<s>', '<s>',] + W + ['</s>']
    if verbose: print(S)
    num_words = len(S)
    PW = 1.0
    for i in range(2,num_words-1): # omit </s> because likelihoods don't have it
        score = lm.score(S[i], [S[i-2], S[i-1]])
        if verbose: print('P({} | {}, {}) = {}'.format(S[i], S[i-2], S[i-1], score))
        PW *= score
    return PW

print('X =', X)
score = P(X, verbose=True)
print('P(X) = {}'.format(score))
print('\nW = CX[1] = {}'.format(CX[1]))
score = P(CX[1], verbose=True)
print('P(W) = {}'.format(score))

X = ['I', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'I', 'wish', 'you', 'where', 'here', '</s>']
P(I | <s>, <s>) = 0.08384932920536636
P(wish | <s>, I) = 0.016923076923076923
P(you | I, wish) = 0.34375
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(X) = 1.1475972685804098e-11

W = CX[1] = ['A', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'A', 'wish', 'you', 'where', 'here', '</s>']
P(A | <s>, <s>) = 0.011996904024767802
P(wish | <s>, A) = 9.937888198757765e-05
P(you | A, wish) = 0.04477611940298508
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(W) = 1.2559652429973822e-15


In [10]:
import math

In [62]:
#EX1
log_priors = []
for W in CX:
    log_priors.append(np.log(P(W)))

log_posteriors = np.array(log_priors)+np.array(np.log(likelihoods))
idx_best_post = np.argmax(log_posteriors)

for i in range(num_candidates):
    best = '<----' if i==idx_best_post else ''
    print('\t'.join(CX[i]), '\tlikelihood={}\tprior={}\tposterior={} {}'
          .format(likelihoods[i], log_priors[i],log_posteriors[i],best))

print('\nThe original sentence was')
print('\t' + ' '.join(X))
print('The right sentence is')
print('\t' + ' '.join(CX[idx_best_post]))

I	wish	you	where	here 	likelihood=0.7737809374999999	prior=-25.190765597926926	posterior=-25.44723206986468 
A	wish	you	where	here 	likelihood=0.005817901785714291	prior=-34.31087200002036	posterior=-39.45768760017987 
a	wish	you	where	here 	likelihood=0.005817901785714291	prior=-36.24830622321093	posterior=-41.39512182337044 
s	wish	you	where	here 	likelihood=0.005817901785714291	prior=-35.70189587142808	posterior=-40.848711471587585 
IV	wish	you	where	here 	likelihood=0.005817901785714291	prior=-inf	posterior=-inf 
If	wish	you	where	here 	likelihood=0.005817901785714291	prior=-34.53940639996945	posterior=-39.68622200012896 
In	wish	you	where	here 	likelihood=0.005817901785714291	prior=-34.85448744660934	posterior=-40.00130304676885 
It	wish	you	where	here 	likelihood=0.005817901785714291	prior=-33.156496136833795	posterior=-38.3033117369933 
I	fish	you	where	here 	likelihood=0.020362656250000017	prior=-40.154760378806564	posterior=-44.0488130104707 
I	with	you	where	here 	likelihood=

<ipython-input-62-328e94caab48>:3: RuntimeWarning: divide by zero encountered in log
  log_priors.append(np.log(P(W)))


In [90]:
#ex4
sentences = ['They danger was here','I fish you where here','her father and herself were let do dine together', 'I will go do sleep', 'He was an bid man', 'He was a had man', 'Only too of us', 'Only too of up','I ill help your', 'I ill help you', 'A will miss you',
             'It was true that her friend ways going only half a mild', 'It was true that her friend was doing only half am mile','You will she me again', 'You will seen my again','I am what your made we', 'I am what your made me',
             'He is got may father', 'He is not may father','then your are nor my friend','then you are nor by friend']
print(len(sentences))
Xs = []
CXs = []
for sentence in sentences:
  X = sentence.split(' ')
  Xs.append(X)
  for x in X:
      assert x in vocabulary, 'All the words in the sentence must belong to the vocabulary, {} doesn\'t'.format(x)

  CX = [X]  # Original sentence

  for i in range(len(X)):
      if X[i] in vocabulary:
          for cw1 in close_words[X[i]]:
              if cw1 != X[i]:
                  C1 = X.copy()
                  C1[i] = cw1
                  CX.append(C1)

                  for j in range(i+1, len(X)):
                      if X[j] in vocabulary and X[j] != cw1:
                          for cw2 in close_words[X[j]]:
                              if cw2 != X[j]:
                                  C2 = C1.copy()
                                  C2[j] = cw2
                                  CX.append(C2)
      else:
          pass
  CXs.append(CX)


21


In [91]:
alpha = 0.95
Likelihoods = []
for CX in CXs:
  likelihoods = []
  for W in CX:
      PXW = 1.0
      #print(X,W)
      for x,w in zip(X,W):
          if w==x:
              pxw = alpha
          else:
              close_to_x = close_words[x] # includes x itself
              pxw = (1-alpha) / (len(close_to_x) - 1) # so we substract 1
          PXW *= pxw
      likelihoods.append(PXW)

  likelihoods = np.array(likelihoods)
  Likelihoods.append(likelihoods)

In [92]:
for i in range(len(CXs)):
  log_priors = []
  for W in CXs[i]:
      log_priors.append(np.log(P(W)))

  log_posteriors = np.array(log_priors)+np.array(np.log(Likelihoods[i]))
  idx_best_post = np.argmax(log_posteriors)

  print('\nThe original sentence was')
  print('\t' + ' '.join(Xs[i]))
  print('The right sentence is')
  print('\t' + ' '.join(CXs[i][idx_best_post]))


The original sentence was
	They danger was here
The right sentence is
	The danger was her

The original sentence was
	I fish you where here
The right sentence is
	I wish you were here

The original sentence was
	her father and herself were let do dine together
The right sentence is
	her father and herself were left to dine together

The original sentence was
	I will go do sleep
The right sentence is
	I will go to sleep

The original sentence was
	He was an bid man
The right sentence is
	He was a bad man

The original sentence was
	He was a had man
The right sentence is
	He was a sad an

The original sentence was
	Only too of us
The right sentence is
	Only to of us

The original sentence was
	Only too of up
The right sentence is
	Only to of us

The original sentence was
	I ill help your
The right sentence is
	I will help you

The original sentence was
	I ill help you
The right sentence is
	I will help you

The original sentence was
	A will miss you
The right sentence is
	I will Miss yo

<ipython-input-92-4610e3dea6fe>:4: RuntimeWarning: divide by zero encountered in log
  log_priors.append(np.log(P(W)))



The original sentence was
	It was true that her friend was doing only half am mile
The right sentence is
	It was true that her friend was going only half a mile

The original sentence was
	You will she me again
The right sentence is
	You will he be again

The original sentence was
	You will seen my again
The right sentence is
	You will see me again

The original sentence was
	I am what your made we
The right sentence is
	I am that you made we

The original sentence was
	I am what your made me
The right sentence is
	I am that you made me

The original sentence was
	He is got may father
The right sentence is
	He is not my father

The original sentence was
	He is not may father
The right sentence is
	He is not my father

The original sentence was
	then your are nor my friend
The right sentence is
	When you are nor my friend

The original sentence was
	then you are nor by friend
The right sentence is
	When you are not by friend


In [93]:
#ex5
sentences = ['The danger was here','then you are not my friend','He is my father','I am what you made me','I will miss you','She was my sister','I will go to sleep','He is a bad man']
Xs = []
CXs = []
for sentence in sentences:
  X = sentence.split(' ')
  Xs.append(X)
  for x in X:
      assert x in vocabulary, 'All the words in the sentence must belong to the vocabulary, {} doesn\'t'.format(x)

  CX = [X]  # Original sentence

  for i in range(len(X)):
      if X[i] in vocabulary:
          for cw1 in close_words[X[i]]:
              if cw1 != X[i]:
                  C1 = X.copy()
                  C1[i] = cw1
                  CX.append(C1)

                  for j in range(i+1, len(X)):
                      if X[j] in vocabulary and X[j] != cw1:
                          for cw2 in close_words[X[j]]:
                              if cw2 != X[j]:
                                  C2 = C1.copy()
                                  C2[j] = cw2
                                  CX.append(C2)
      else:
          pass
  CXs.append(CX)


In [94]:
alpha = 0.95
Likelihoods = []
for CX in CXs:
  likelihoods = []
  for W in CX:
      PXW = 1.0
      #print(X,W)
      for x,w in zip(X,W):
          if w==x:
              pxw = alpha
          else:
              close_to_x = close_words[x] # includes x itself
              pxw = (1-alpha) / (len(close_to_x) - 1) # so we substract 1
          PXW *= pxw
      likelihoods.append(PXW)

  likelihoods = np.array(likelihoods)
  Likelihoods.append(likelihoods)


for i in range(len(CXs)):
  log_priors = []
  for W in CXs[i]:
      log_priors.append(np.log(P(W)))

  log_posteriors = np.array(log_priors)+np.array(np.log(Likelihoods[i]))
  idx_best_post = np.argmax(log_posteriors)

  print('\nThe original sentence was')
  print('\t' + ' '.join(Xs[i]))
  print('The right sentence is')
  print('\t' + ' '.join(CXs[i][idx_best_post]))


The original sentence was
	The danger was here
The right sentence is
	The danger as her

The original sentence was
	then you are not my friend
The right sentence is
	When you are got my friend

The original sentence was
	He is my father
The right sentence is
	He is my father

The original sentence was
	I am what you made me
The right sentence is
	I am that you make me

The original sentence was
	I will miss you
The right sentence is
	I will Miss you

The original sentence was
	She was my sister
The right sentence is
	She was my sister

The original sentence was
	I will go to sleep
The right sentence is
	I will go to sleep

The original sentence was
	He is a bad man
The right sentence is
	He is a bad man


<ipython-input-94-793cd4a22b85>:24: RuntimeWarning: divide by zero encountered in log
  log_priors.append(np.log(P(W)))


That's it! Now, let's try this code and do some modifications:
1. change probabilities by log of probabilities to get scores more easy to interpret
1. relax the assumption that in a sentence there's at most 1 mispelled word, what if we suppose there may be at most 2 ?
1. and what is the effect of $\alpha$ ? change it to 0.9, 0.8...
1. make a long (>20) list of sentences each with 1 or 2 mispelled words to assess the speller : does it really work ?
1. same for a second list of sentences withouth any spelling error (but different from any one in the selected part of the book), are there any errors ?
1. extra points: what if you change the tri-gram language model by the neural network of exercise 2 ?